In [57]:
import tensorflow as tf
import numpy as np
import os
import time
import urllib.request
import re

In [58]:
url = "https://www.gutenberg.org/files/11/11-0.txt"
file = urllib.request.urlopen(url)
text = [line.decode('utf-8') for line in file]
text = ''.join(text)
text = re.sub(' +',' ',text)
text = re.sub(r'[^A-Za-z.,!\r ]+', '', text)
text = text[1150:]
text[:200]

'TER VII. A Mad TeaParty\r CHAPTER VIII. The Queens CroquetGround\r CHAPTER IX. The Mock Turtles Story\r CHAPTER X. The Lobster Quadrille\r CHAPTER XI. Who Stole the Tarts\r CHAPTER XII. Alices Evidence\r\r\r\r'

In [59]:
vocab = sorted(set(text))
",".join(vocab)

'\r, ,!,,,.,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z'

In [60]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
[f"{char} = {i}" for char, i in zip(char2idx, range(20))]

['\r = 0',
 '  = 1',
 '! = 2',
 ', = 3',
 '. = 4',
 'A = 5',
 'B = 6',
 'C = 7',
 'D = 8',
 'E = 9',
 'F = 10',
 'G = 11',
 'H = 12',
 'I = 13',
 'J = 14',
 'K = 15',
 'L = 16',
 'M = 17',
 'N = 18',
 'O = 19']

In [61]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
[idx2char[i.numpy()] for i in char_dataset.take(5)]

['T', 'E', 'R', ' ', 'V']

In [62]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
[repr(''.join(idx2char[item.numpy()])) for item in sequences.take(5)]

["'TER VII. A Mad TeaParty\\r CHAPTER VIII. The Queens CroquetGround\\r CHAPTER IX. The Mock Turtles Story\\r '",
 "'CHAPTER X. The Lobster Quadrille\\r CHAPTER XI. Who Stole the Tarts\\r CHAPTER XII. Alices Evidence\\r\\r\\r\\r\\rC'",
 "'HAPTER I.\\rDown the RabbitHole\\r\\r\\rAlice was beginning to get very tired of sitting by her sister on the'",
 "'\\rbank, and of having nothing to do once or twice she had peeped into\\rthe book her sister was reading,'",
 "' but it had no pictures or\\rconversations in it, and what is the use of a book, thought Alice\\rwithout '"]

In [63]:
@tf.autograph.experimental.do_not_convert
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [64]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'TER VII. A Mad TeaParty\r CHAPTER VIII. The Queens CroquetGround\r CHAPTER IX. The Mock Turtles Story\r'
Target data: 'ER VII. A Mad TeaParty\r CHAPTER VIII. The Queens CroquetGround\r CHAPTER IX. The Mock Turtles Story\r '


In [65]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 24 ('T')
  expected output: 9 ('E')
Step    1
  input: 9 ('E')
  expected output: 22 ('R')
Step    2
  input: 22 ('R')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 26 ('V')
Step    4
  input: 26 ('V')
  expected output: 13 ('I')


In [66]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
rnn_units_2 = 512

In [67]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[BATCH_SIZE, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),  
  tf.keras.layers.Dense(vocab_size)
])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (64, None, 256)           14592     
                                                                 
 gru_18 (GRU)                (64, None, 1024)          3938304   
                                                                 
 gru_19 (GRU)                (64, None, 512)           2362368   
                                                                 
 dense_9 (Dense)             (64, None, 57)            29241     
                                                                 
Total params: 6,344,505
Trainable params: 6,344,505
Non-trainable params: 0
_________________________________________________________________


In [68]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [69]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [70]:
history = model.fit(dataset, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
24/24 [==============================] - 24s 954ms/step - loss: 3.2620
Epoch 2/50
24/24 [==============================] - 23s 938ms/step - loss: 2.7040
Epoch 3/50
24/24 [==============================] - 23s 939ms/step - loss: 2.3630
Epoch 4/50
24/24 [==============================] - 23s 940ms/step - loss: 2.1917
Epoch 5/50
24/24 [==============================] - 23s 939ms/step - loss: 2.0469
Epoch 6/50
24/24 [==============================] - 23s 939ms/step - loss: 1.9155
Epoch 7/50
24/24 [==============================] - 23s 939ms/step - loss: 1.8009
Epoch 8/50
24/24 [==============================] - 23s 941ms/step - loss: 1.6919
Epoch 9/50
24/24 [==============================] - 23s 939ms/step - loss: 1.5983
Epoch 10/50
24/24 [==============================] - 23s 941ms/step - loss: 1.5063
Epoch 11/50
24/24 [==============================] - 23s 939ms/step - loss: 1.4248
Epoch 12/50
24/24 [==============================] - 23s 938ms/step - loss: 1.3498
Epoch 13/50
2

In [71]:
def generate_text(model, start_string, temp, gen_chars):     
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)  
  text_generated = []
  model.reset_states()
  for i in range(gen_chars):
    predictions = model(input_eval)      
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temp
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])  
  return (start_string + ''.join(text_generated))

In [72]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[1, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.Dense(vocab_size)
])
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[1, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.Dense(vocab_size)
])
model.summary()
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (1, None, 256)            14592     
                                                                 
 gru_22 (GRU)                (1, None, 1024)           3938304   
                                                                 
 gru_23 (GRU)                (1, None, 512)            2362368   
                                                                 
 dense_11 (Dense)            (1, None, 57)             29241     
                                                                 
Total params: 6,344,505
Trainable params: 6,344,505
Non-trainable params: 0
_________________________________________________________________


In [81]:
generate_text(model, u"Alice said", 1.0, 200)

'Alice said to the Dormouse, in a shrill, passionate voice. Would\ryou like cats if you were me\r\rWell, perhaps not, said the Mouse. I proceed. Edwin anxy direction, with a treelictures or conversations\r\rSo she wa'